# Branch and Bound from scratch

In [1]:
import numpy as np
from scipy.optimize import linprog

In [2]:
from bnb import BranchAndBound

## Problem 1

$$
\begin{align}
    \text{maximize}~ \;\; & 5 x_{1} + 4 x_{2} \\
    \text{subject to}~ \;\; & 2 x_{1} + 3 x_{2} \leq 12 \\
    & 2 x_{1} + x_{2} \leq 6 \\
    & x_{i} \geq 0 & \forall \; i \in \{  1, 2 \} \\
    & x_{i} \in \mathbb{Z} & \forall \; i \in \{  1, 2 \}
\end{align}
$$

In [3]:
c = np.array([-5.0, -4.0])

A_ub = np.array(
    [[2.0, 3.0],
     [2.0, 1.0]]
)
b_ub = np.array([12.0, 6.0])

In [4]:
bb = BranchAndBound(branching_rule="frac", node_rule="bfs")
sol = bb(c, A_ub=A_ub, b_ub=b_ub, lb=0, ub=np.inf, integrality=None, verbose=False, mip_gap=1e-6, max_iter=1000)
print(sol)

        message: Optimal integer solution found
        success: True
         status: 0
            fun: -18.0
              x: [ 2.000e+00  2.000e+00]
            nit: 0
          lower:  residual: [ 0.000e+00  2.000e+00]
                 marginals: [ 3.000e+00  0.000e+00]
          upper:  residual: [       inf        inf]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 2.000e+00  0.000e+00]
                 marginals: [-0.000e+00 -4.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0
       explored: 4
       fathomed: 0
     best_bound: -18.0


In [5]:
sol_scipy = linprog(c, A_ub=A_ub, b_ub=b_ub, integrality=np.ones(2))
print(sol_scipy)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -18.0
              x: [ 2.000e+00  2.000e+00]
            nit: -1
          lower:  residual: [ 2.000e+00  2.000e+00]
                 marginals: [ 0.000e+00  0.000e+00]
          upper:  residual: [       inf        inf]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 2.000e+00 -8.882e-16]
                 marginals: [ 0.000e+00  0.000e+00]
 mip_node_count: 1
 mip_dual_bound: -18.0
        mip_gap: 0.0


## Problem 2

$$
\begin{align}
    \text{max} \quad & \sum_{i \in I}{c_{i} x_{i}} \\
    \text{s.t.} \quad & \sum_{i \in I}{w_{i} x_{i}} \leq k_{w} & \forall ~i \in I\\
    & \sum_{i \in I}{v_{i} x_{i}} \leq k_{v} & \forall ~i \in I\\
    & x_{i} \in \left \{ 0, 1 \right \} & \forall ~i \in I\\
\end{align}
$$

In [6]:
# Set random seed
np.random.seed(42)
N = 10

# Weight associated with each item
w = np.random.normal(loc=5.0, scale=1.0, size=N).clip(0.5, 10.0)
v = np.random.normal(loc=6.0, scale=2.0, size=N).clip(0.5, 10.0)

# Price associated with each item
c = -np.random.normal(loc=10.0, scale=1.0, size=N).clip(0.5, 20.0)

# knapsack capacity
kw = 21.0
kv = 22.0

A_ub = np.atleast_2d([w, v])
b_ub = np.array([kw, kv])

In [7]:
bb = BranchAndBound(branching_rule="frac", node_rule="bfs")
sol = bb(c, A_ub=A_ub, b_ub=b_ub, lb=0, ub=1, integrality=None, verbose=False, mip_gap=1e-4, max_iter=1000)
print(sol)

        message: Optimal integer solution found
        success: True
         status: 0
            fun: -41.726493076490556
              x: [ 1.000e+00  1.000e+00  0.000e+00  0.000e+00  0.000e+00
                   1.000e+00  0.000e+00  1.000e+00  0.000e+00  0.000e+00]
            nit: 0
          lower:  residual: [ 1.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  1.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          upper:  residual: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
                 marginals: [-1.147e+01 -9.774e+00 -1.007e+01 -8.575e+00
                             -9.456e+00 -

In [8]:
sol_scipy = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=(0, 1), integrality=np.ones(N))
print(sol_scipy)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -41.726493076490556
              x: [ 1.000e+00  1.000e+00  0.000e+00  0.000e+00  0.000e+00
                   1.000e+00  0.000e+00  1.000e+00  0.000e+00  0.000e+00]
            nit: -1
          lower:  residual: [ 1.000e+00  1.000e+00  0.000e+00  0.000e+00
                              0.000e+00  1.000e+00  0.000e+00  1.000e+00
                              0.000e+00  0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          upper:  residual: [ 0.000e+00  0.000e+00  1.000e+00  1.000e+00
                              1.000e+00  0.000e+00  1.000e+00  0.000e+00
                              1.000e+00  1.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
       